In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [2]:
train = pd.read_csv("melon_train.csv")
test = pd.read_csv("melon_test.csv")

print(train.shape)
print(test.shape)

(2988, 8)
(996, 8)


In [3]:
train

,Unnamed: 0,song,artist,comment,comment_date,jjim,rating,label
0,5544,조금 취했어,임재현,가만보면 멜론 이새1퀴들이 더 나쁜넘들임ㅋㅋㅋ 사재기논란은 몇년전부터 계속 나왔...,2019.09.27 01:50,"3,484",1.0,1
1,4980,조금 취했어,임재현,그만 좀 취하자.. 외국인이 보면 대한민국 사람들은 365일 애인이랑 헤어지고 ...,2019.11.27 12:22,"3,484",1.0,1
2,141,항해,AKMU (악동뮤지션),더러워진 차트에서 진짜 순수 대중픽으로 1위 유지하는 거 왤케 멋있냐 ㅠㅠㅠㅠ ...,2019.10.21 23:19,"50,786",4.7,0
3,1965,항해,AKMU (악동뮤지션),악뮤는 정말 천재인거같아요 대단한거같아요,2019.10.02 15:55,"50,786",4.7,0
4,5594,조금 취했어,임재현,가요대전 나왔겠죠? 올한해 음원차트 씹어드셨으니까?,2019.12.26 04:39,"3,484",1.0,1
...,...,...,...,...,...,...,...,...
2983,5453,조금 취했어,임재현,아직도 임재현을 쉴드치는 사람들이 있다고? 진짜 궁금한데 돈 얼마받고 쉴드치고 ...,2019.11.24 17:02,"3,484",1.0,1
2984,1472,항해,AKMU (악동뮤지션),올해의 노래다,2019.10.25 13:02,"50,786",4.7,0
2985,698,항해,AKMU (악동뮤지션),항해야 너 3주된 앨범 맞지??? 어제나온 앨범아니지?? 아니 뭐 .. 그래프가...,2019.10.14 16:09,"50,786",4.7,0
2986,63,항해,AKMU (악동뮤지션),한 가지 확신할 수 있는 점이거 명반 딱지 붙음.아니 붙을 수 밖에 없음 ㅋㅋ,2019.10.16 14:18,"50,786",4.7,0


## 1. 전처리 - Train Set

In [4]:
# 리뷰 문장 추출
sentences = train['comment']
sentences

0         가만보면 멜론 이새1퀴들이 더 나쁜넘들임ㅋㅋㅋ 사재기논란은 몇년전부터 계속 나왔...
1         그만 좀 취하자.. 외국인이 보면 대한민국 사람들은 365일 애인이랑 헤어지고 ...
2         더러워진 차트에서 진짜 순수 대중픽으로 1위 유지하는 거 왤케 멋있냐 ㅠㅠㅠㅠ ...
3                                 악뮤는 정말 천재인거같아요 대단한거같아요 
4                           가요대전 나왔겠죠? 올한해 음원차트 씹어드셨으니까? 
                              ...                        
2983      아직도 임재현을 쉴드치는 사람들이 있다고? 진짜 궁금한데 돈 얼마받고 쉴드치고 ...
2984                                             올해의 노래다 
2985      항해야 너 3주된 앨범 맞지??? 어제나온 앨범아니지?? 아니 뭐 .. 그래프가...
2986         한 가지 확신할 수 있는 점이거 명반 딱지 붙음.아니 붙을 수 밖에 없음 ㅋㅋ 
2987      아니 왜 악뮤 밑으로 내려가라고 그러세요 다들?저 멀리 차트 밖으로 내려가라고 ...
Name: comment, Length: 2988, dtype: object

In [5]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS]   가만보면 멜론 이새1퀴들이 더 나쁜넘들임ㅋㅋㅋ 사재기논란은 몇년전부터 계속 나왔는데 머하나 달라진게 없음ㅇㅇ거의 음원시장에선 가장 큰 기업이니 간땡이가 부은거지ㅉ 결국 이제 가수 누가나오든 대놓고 기계돌리는 사태까지 일어남  [SEP]',
 '[CLS]   그만 좀 취하자.. 외국인이 보면 대한민국 사람들은 365일 애인이랑 헤어지고 술먹고 전화거는 알코올중독 진상으로 보겠다ㅠㅠ  [SEP]',
 '[CLS]   더러워진 차트에서 진짜 순수 대중픽으로 1위 유지하는 거 왤케 멋있냐 ㅠㅠㅠㅠ 11월부터는 아이유랑 같이 1,2위 차트 채워 줬으면,,,  [SEP]',
 '[CLS]   악뮤는 정말 천재인거같아요 대단한거같아요  [SEP]',
 '[CLS]   가요대전 나왔겠죠? 올한해 음원차트 씹어드셨으니까?  [SEP]',
 '[CLS]   멜로디가 너무 익숙한데 유명한 발라드 여러개 섞어 듣는 느낌은 나만 느끼는 건가  [SEP]',
 '[CLS]   전 재계약 좋네유ㅠㅠㅠ앞으로도 좋은 음악 들려주세용 :-)  [SEP]',
 '[CLS]   와이지 불매한다는 빡머갈들은 갑자기 회사 망하면 본인 인생 끝낼 거임?  [SEP]',
 '[CLS]   정직한 가수가 되시길 재생 다운로드 곡명 바랍니다 아티스트명 호란 호란 앨범명 바랍니다  [SEP]',
 '[CLS]   악뮤는 진짜 대단하다..  [SEP]']

In [6]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 1, 0, ..., 0, 0, 1])

In [7]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS]   가만보면 멜론 이새1퀴들이 더 나쁜넘들임ㅋㅋㅋ 사재기논란은 몇년전부터 계속 나왔는데 머하나 달라진게 없음ㅇㅇ거의 음원시장에선 가장 큰 기업이니 간땡이가 부은거지ㅉ 결국 이제 가수 누가나오든 대놓고 기계돌리는 사태까지 일어남  [SEP]
['[CLS]', '가', '##만', '##보', '##면', '멜', '##론', '이', '##새', '##1', '##퀴', '##들이', '더', '[UNK]', '사', '##재', '##기', '##논', '##란', '##은', '몇', '##년', '##전', '##부터', '계속', '나', '##왔', '##는데', '머', '##하', '##나', '달', '##라', '##진', '##게', '[UNK]', '음', '##원', '##시', '##장에', '##선', '가장', '큰', '기', '##업', '##이', '##니', '[UNK]', '[UNK]', '결국', '이', '##제', '가수', '누', '##가', '##나', '##오', '##든', '대', '##놓', '##고', '기', '##계', '##돌', '##리는', '사', '##태', '##까지', '일', '##어', '##남', '[SEP]']


In [8]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8843,  19105,  30005,  14867,   9275,  42769,   9638,
       119031,  10759, 119322,  20173,   9074,    100,   9405,  36210,
        12310, 118743,  49919,  10892,   9282,  10954,  16617,  17655,
        77039,   8982, 119163,  41850,   9265,  35506,  16439,   9061,
        17342,  18623,  14153,    100,   9634,  14279,  14040,  85903,
        18471,  22224,   9835,   8932,  26784,  10739,  25503,    100,
          100,  50342,   9638,  17730, 108399,   9032,  11287,  16439,
        28188,  90537,   9069, 118748,  11664,   8932,  21611, 118794,
        26344,   9405,  83616,  18382,   9641,  12965,  37004,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [9]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([ 101,  100, 1869,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(0)
tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [11]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 2. 전처리 - Test set

In [12]:
# 리뷰 문장 추출
sentences = test['comment']
sentences[:10]

0                               가온차트뮤직어워드에 나오네.. 철판오져 
1      찬혁님 컴퓨터에 저장된 곡만 150곡 넘는다는데 그걸로 나중에 윤종신처럼 월간악...
2              리뷰창정화해드릴게요ㅠㅠ2년만에 컴백인데 저런 ㅇㄱㄹ들이나 몰려오고ㅠㅠ 
3                            논란 때문인가 겁 엄청 먹고 무대 나왔네.. 
4                                악플들, 도배글들 신고 눌러주세요~~ 
5                         명반 업데이트 한번 더 하면 무조건 붙어있을 거임 
6                                     매일 들어도 질리지가 않는다 
7                                               누구세요? 
8                 대체 2soo 는 또 누구임? 내가 아는 이수는 엠맥밖에 없는데 
9                                   술 이별 징징징징 발라드..어휴 
Name: comment, dtype: object

In [13]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS]   가온차트뮤직어워드에 나오네.. 철판오져  [SEP]',
 '[CLS]   찬혁님 컴퓨터에 저장된 곡만 150곡 넘는다는데 그걸로 나중에 윤종신처럼 월간악뮤 이런거 해주면 행복할 것 같다...  [SEP]',
 '[CLS]   리뷰창정화해드릴게요ㅠㅠ2년만에 컴백인데 저런 ㅇㄱㄹ들이나 몰려오고ㅠㅠ  [SEP]',
 '[CLS]   논란 때문인가 겁 엄청 먹고 무대 나왔네..  [SEP]',
 '[CLS]   악플들, 도배글들 신고 눌러주세요~~  [SEP]',
 '[CLS]   명반 업데이트 한번 더 하면 무조건 붙어있을 거임  [SEP]',
 '[CLS]   매일 들어도 질리지가 않는다  [SEP]',
 '[CLS]   누구세요?  [SEP]',
 '[CLS]   대체 2soo 는 또 누구임? 내가 아는 이수는 엠맥밖에 없는데  [SEP]',
 '[CLS]   술 이별 징징징징 발라드..어휴  [SEP]']

In [14]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,

In [15]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS]   가온차트뮤직어워드에 나오네.. 철판오져  [SEP]
['[CLS]', '가', '##온', '##차', '##트', '##뮤', '##직', '##어', '##워', '##드', '##에', '나', '##오', '##네', '.', '.', '철', '##판', '##오', '##져', '[SEP]']


In [16]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8843,  37093,  23466,  15184, 118953,  33077,  12965,
        69592,  15001,  10530,   8982,  28188,  77884,    119,    119,
         9747,  33323,  28188,  41584,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [17]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [18]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8843,  37093,  23466,  15184, 118953,  33077,  12965,  69592,
         15001,  10530,   8982,  28188,  77884,    119,    119,   9747,  33323,
         28188,  41584,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [19]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 3. 모델 생성

In [20]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

SystemError: GPU device not found

In [21]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [22]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [24]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 3

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

## 4. 모델학습

In [25]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [26]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # print(step, batch)
        # 경과 정보 표시
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch    10  of     85.    Elapsed: 0:06:52.
  Batch    20  of     85.    Elapsed: 0:13:13.
  Batch    30  of     85.    Elapsed: 0:19:16.
  Batch    40  of     85.    Elapsed: 0:25:31.
  Batch    50  of     85.    Elapsed: 0:31:12.
  Batch    60  of     85.    Elapsed: 0:37:15.
  Batch    70  of     85.    Elapsed: 0:43:03.
  Batch    80  of     85.    Elapsed: 0:48:33.

  Average training loss: 0.61
  Training epcoh took: 0:50:47

Running Validation...
  Accuracy: 0.88
  Validation took: 0:01:16

======== Epoch 2 / 3 ========
Training...
  Batch    10  of     85.    Elapsed: 0:05:56.
  Batch    20  of     85.    Elapsed: 0:12:03.
  Batch    30  of     85.    Elapsed: 0:17:46.
  Batch    40  of     85.    Elapsed: 0:23:36.
  Batch    50  of     85.    Elapsed: 0:29:44.
  Batch    60  of     85.    Elapsed: 0:35:51.
  Batch    70  of     85.    Elapsed: 0:42:13.
  Batch    80  of     85.    Elapsed: 0:48:06.

  Average training loss: 0.35
  

## Test set 평가

In [28]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 10 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch    10  of     32.    Elapsed: 0:01:36.
  Batch    20  of     32.    Elapsed: 0:03:11.
  Batch    30  of     32.    Elapsed: 0:04:44.

Accuracy: 0.89
Test took: 0:04:56


# 새로운 문장 테스트

In [29]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [30]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [37]:
logits = test_sentences(['전곡이 다 좋자나 타이틀만 주로 들어왔는데 전곡 돌려듣는거 진짜 오랜만이다 악뮤 사랑 많이 받아도 늘 변함없는 모습으로 오래오래 좋은 음악해주길수록곡중엔 고래 특히 취저뭔가 독특한 분위기의 곡임뱃노래 작별인사 달 밤끝없는밤도 '])

print(logits)
print(np.argmax(logits))

[[ 1.7011026 -2.2667449]]
0


In [38]:
logits = test_sentences(['어그로들 이때다 싶어 달려드는거 진짜 꼴불견이다안봐도 3일안에 다시 1위 탈환할것같구만;; '])

print(logits)
print(np.argmax(logits))

[[ 0.50833285 -1.0564018 ]]
0


In [35]:
logits = test_sentences(['프사로 사기치고 노래로 사기치고 시상식 사기치고 '])

print(logits)
print(np.argmax(logits))

[[-1.535981  1.493666]]
1


In [49]:
logits = test_sentences(['술 이별 징징징징 발라드..어휴 '])

print(logits)
print(np.argmax(logits))

[[-0.33938256  0.2064023 ]]
1


In [86]:
yes = ['술 이별 징징징징 발라드..어휴 ', '프사로 사기치고 노래로 사기치고 시상식 사기치고', '내 친구들이 뭣도 모르고 이 노래 막 노래방에서 부르는거 종나 ♩♪이었음 노래방순위도 높더라?ㅋㅌㅌㅌ 역겨워..ㅌㅋㅋㅋㅋ 니가 있을곳 아니니까 양심있게 내려와라', '  이게 사재기가 아니면 도대체 이노래 Inst가 검색 노래 인기 1,2등을 나란히 하고있는걸까 ']
no = ['매일 들어도 질리지가 않는다', '와~ 다시 음원강자 ㅍㅋ 제끼고 1위 탈환 했네~ 역시 대중픽! 악뮤는 사랑입니다. ', '근데 타이틀곡 선정은 잘함 전곡이 다 좋지만 요즘같이 발라드 극강세를 보이는 차트에서 대중성을 잡을만한 발라드곡이니... ', '악뮤가 진짜 기계 여럿 막아준다.. ', '사재기도 못따라오는 클라스... ', '멜론 15위까지 볼수가 없는데 그와중에 기계 이겨주셔서 감사해요', '설마 악뮤 노래에 사재기 소리하는 정신나간 사람들이 있었다니... ', '악뮤한테 사재기 드립은 좀;;', '팩트는 악뮤가 사재기면 다른가수 다 사재기임 ']

In [87]:
for i in range(len(yes)):
    print([yes[i]])
    logits = test_sentences([yes[i]])
    print('결과', np.argmax(logits))    

['술 이별 징징징징 발라드..어휴 ']
결과 1
['프사로 사기치고 노래로 사기치고 시상식 사기치고']
결과 1
['내 친구들이 뭣도 모르고 이 노래 막 노래방에서 부르는거 종나 ♩♪이었음 노래방순위도 높더라?ㅋㅌㅌㅌ 역겨워..ㅌㅋㅋㅋㅋ 니가 있을곳 아니니까 양심있게 내려와라']
결과 1
['  이게 사재기가 아니면 도대체 이노래 Inst가 검색 노래 인기 1,2등을 나란히 하고있는걸까 ']
결과 1


In [88]:
for i in range(len(no)):
    print([no[i]])
    logits = test_sentences([no[i]])
    print('결과', np.argmax(logits)) 

['매일 들어도 질리지가 않는다']
결과 0
['와~ 다시 음원강자 ㅍㅋ 제끼고 1위 탈환 했네~ 역시 대중픽! 악뮤는 사랑입니다. ']
결과 0
['근데 타이틀곡 선정은 잘함 전곡이 다 좋지만 요즘같이 발라드 극강세를 보이는 차트에서 대중성을 잡을만한 발라드곡이니... ']
결과 0
['악뮤가 진짜 기계 여럿 막아준다.. ']
결과 0
['사재기도 못따라오는 클라스... ']
결과 1
['멜론 15위까지 볼수가 없는데 그와중에 기계 이겨주셔서 감사해요']
결과 1
['설마 악뮤 노래에 사재기 소리하는 정신나간 사람들이 있었다니... ']
결과 0
['악뮤한테 사재기 드립은 좀;;']
결과 0
['팩트는 악뮤가 사재기면 다른가수 다 사재기임 ']
결과 0
